# Walmart: predict weekly sales
#### <i>Author: Delphine César<i>

## Table of contents

<ul>
   <li><a href="#import">I - Import of librairies and dataset</a></li>
   <li><a href="#info">II - Dataset information</a></li>
   <li><a href="#eda">III - EDA</a></li>
   <li><a href="#engineering">IV - Data engineering</a></li>
   <li><a href="#ml">V - Machine Learning</a></li>
      <ul>
         <li><a href="#preprocessing">1 - Preprocessing</a></li>
         <li><a href="#train">2 - Train model</a></li>
         <li><a href="#performance">3 - Performance assessment</a></li>
         <li><a href="#regularization">4 - Regularization</a></li>
            <ul>
               <li><a href="#ridge">a - Ridge</a></li>
               <li><a href="#lasso">b - Lasso</a></li>
            </ul>
         <li><a href="#optimization">5 - Hyperparameter optimization</a></li>
      </ul>
</ul>

<a id='import'></a>
### I - Import of librairies and dataset

In [1]:
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, cross_val_score


In [2]:
dataset = pd.read_csv("Walmart_Store_sales.csv")

<a id='info'></a>
### II - Dataset information

In [3]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

print("Columns type")
display(dataset.info())

Number of rows : 150

Number of columns : 8

Display of dataset: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092



Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

Columns type
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         150 non-null    float64
 1   Date          132 non-null    object 
 2   Weekly_Sales  136 non-null    float64
 3   Holiday_Flag  138 non-null    float64
 4   Temperature   132 non-null    float64
 5   Fuel_Price    136 non-null    float64
 6   CPI           138 non-null    float64
 7   Unemployment  135 non-null    float64
dtypes: float64(7), object(1)
memory usage: 9.5+ KB


None

There are some missing values on the target, we will need to drop some lines

<a id='eda'></a>
### III - EDA

In [4]:
features = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Holiday_Flag', 'Store']
for i in range(len(features)):
    fig = px.histogram(dataset[features[i]])
    fig.show()

There are obviously some outliers that we will need to drop

In [5]:
# Correlation matrix
corr_matrix = dataset.loc[:,features].corr().round(2)
fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())
fig.show()

Strong correlations :
- CPI / Store
- CPI /Unemployement
- CPI / Weekly_Sales
- Temperature / Store

<a id='engineering'></a>
### IV - Data engineering

In [6]:
# Drop lines containing outliers
cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']

print('Dropping outliers in Temperature, Fuel Price, CPI and Unemployment...')

for i in dataset[cols]:
    outlier_min = dataset[i].mean() - 3*dataset[i].std()
    outlier_max = dataset[i].mean() + 3*dataset[i].std()
    to_keep = (dataset[i] > outlier_min) & (dataset[i] < outlier_max)
    dataset = dataset.loc[to_keep,:]

print('Done. Number of lines remaining : ', dataset.shape[0])

Dropping outliers in Temperature, Fuel Price, CPI and Unemployment...
Done. Number of lines remaining :  98


In [7]:
# Drop null lines for weekly sales 
mask = dataset["Weekly_Sales"].isnull()
wsales_drop = dataset.loc[mask,:].index
dataset = dataset.drop(wsales_drop, axis=0)
print('Done. Number of lines remaining : ', dataset.shape[0])
print()

Done. Number of lines remaining :  90



In [8]:
# Convert datet into datetime and create new columns : year, month, day, dayofweek
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset['Year'] = dataset['Date'].apply(lambda x : x.year)
dataset['Month'] = dataset['Date'].apply(lambda x : x.month)
dataset['Day'] = dataset['Date'].apply(lambda x : x.day)
dataset['DayOfWeek'] = dataset['Date'].apply(lambda x : x.dayofweek)

dataset = dataset.drop(['Date'], axis=1)
dataset.head()

/var/folders/h4/0zx733cx5434zvycckpxxfyh0000gn/T/ipykernel_98249/731695249.py:2: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,2011.0,6.0,3.0,4.0
7,20.0,2203523.20,0.0,39.93,3.617,213.023622,6.961,2012.0,2.0,3.0,4.0


In [9]:
dataset['DayOfWeek'].value_counts()

DayOfWeek
4.0    80
Name: count, dtype: int64

Since we only have Fridays in the dataset, we remove the column

In [10]:
dataset = dataset.drop('DayOfWeek', axis=1)
print('Done. Number of lines remaining : ', dataset.shape[0])
print()

Done. Number of lines remaining :  90



<a id='ml'></a>
### V - Machine Learning

<a id='preprocessing'></a>
##### 1 - Preprocessing

In [11]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'

print("Separating labels from features...")
Y = dataset.loc[:,target_name]
X = dataset.drop(target_name, axis = 1) # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()

Separating labels from features...
...Done.
0    1572117.54
1    1807545.43
4    1644470.66
6     695396.19
7    2203523.20
Name: Weekly_Sales, dtype: float64

   Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment   
0    6.0           NaN        59.61       3.045  214.777523         6.858  \
1   13.0           0.0        42.38       3.435  128.616064         7.470   
4    6.0           0.0        78.89       2.759  212.412888         7.092   
6   15.0           0.0        69.80       4.069  134.855161         7.658   
7   20.0           0.0        39.93       3.617  213.023622         6.961   

     Year  Month   Day  
0  2011.0    2.0  18.0  
1  2011.0    3.0  25.0  
4  2010.0    5.0  28.0  
6  2011.0    6.0   3.0  
7  2012.0    2.0   3.0  



In [12]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [13]:
# Create pipeline for numeric features
numeric_features = ["Temperature", "Fuel_Price", "CPI", "Unemployment", "Year", "Month", "Day"]
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag'] 
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))
    ])

In [14]:
# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment   
75    20.0           0.0        75.17       2.808  204.567546         7.856  \
99    13.0           0.0        78.82       2.814  126.139200         7.951   
120    8.0           0.0        75.32       2.582  214.878556         6.315   
110   20.0           1.0        28.85       3.179  204.643227         7.484   
125    3.0           0.0        63.91       3.308  221.643285         7.197   

       Year  Month   Day  
75   2010.0    6.0  25.0  
99   2010.0    7.0   2.0  
120  2010.0    9.0  17.0  
110  2010.0   12.0  31.0  
125  2011.0   11.0  18.0  
...Done.
[[ 0.70516394 -1.13980519  0.62142729  0.48772018 -1.17754396 -0.15325478
   1.05677123  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.          0.        ]
 [ 0.91740363

<a id='train'></a>
##### 2 - Train model

In [15]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


<a id='performance'></a>
##### 3 - Performance assessment

In [16]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[1886875.47947311 2085608.10309974  960983.42485966 1891028.52273137
  455852.3975955   239850.06904666  450005.08210372 1601037.16887773
  380636.14871889 1366103.31115122 1529546.77222228  552289.66675267
 1554758.51963516  925658.72850131  439430.20192293  422305.58590419
 2201107.29985495 1398713.67996663 1155561.19548985 2036560.68365877
 1166083.03245016 1979214.32894453  487374.27239462  464870.779243
 1566795.65739441  448520.17041337 1569607.94        659653.35829309
 2148536.12050664  658169.49893893 1412395.68326784 1957267.96296093
  503901.5695774   468173.55469283  839933.59663903  968746.69092956
 1988466.2539934  1972592.11165359 1295773.34339203  588725.15112222
 2438646.79914555  626588.20993885  458832.90026777 1525721.42269739
 2084543.29435928 1094672.86960076  450061.72446215 1911202.89984852
 1788692.64077817 1519898.81104241 1019936.66152967 2040807.20779552
 1525874.63953745  469652.98296548  510731.27846201  344130.19714

In [17]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[2088358.14841338 1317918.32389674 1886619.36627509 1400947.49493383
 1655889.79840001  932025.51386175  892813.83453881 2446576.02506422
  867462.0026819  1919691.06608086 2165464.0458549  2012359.3884904
  399859.36301349  416872.47593106 1127302.23032016 2046845.54585066
  754657.99818688 1559710.37490701]



In [18]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

R2 score on training set :  0.9817442273614119
R2 score on test set :  0.9470437168592518


In [19]:
scores = cross_val_score(regressor,X_train, Y_train, cv=10)
avg = scores.mean()
std = scores.std()
print('Cross-validated accuracy : {}\nstandard deviation : {}'.format(avg, std))

Cross-validated accuracy : 0.8859773752658132
standard deviation : 0.1880637692167868


When performing cross-validation, we can see that the score can vary by approximately 0.18. Both our train test and test set are in this range which means that our model, is already quite good.

<a id='regularization'></a>
##### 4 - Regularization

Ridge and Lasso both help avoid overfitting: 
- Ridge reduces coefficient values
- Lasso selects the most important variables by removing the coefficient of less significant variables

<a id='ridge'></a>
a - Ridge

In [20]:
ridge = Ridge()
print(ridge)
ridge.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", ridge.score(X_train, Y_train))
print("R2 score on test set : ", ridge.score(X_test, Y_test))

Ridge()
R2 score on training set :  0.9260810166547854
R2 score on test set :  0.8750851164766239


<a id='lasso'></a>
a - Lasso

In [21]:
lasso = Lasso()
print(lasso)
lasso.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", lasso.score(X_train, Y_train))
print("R2 score on test set : ", lasso.score(X_test, Y_test))

Lasso()
R2 score on training set :  0.9811970791169302
R2 score on test set :  0.954597266827667


/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.042e+11, tolerance: 3.247e+09



<a id='optimization'></a>
##### 5 - Hyperparameter optimization

In [22]:
# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100]
}
best_ridge = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
best_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 0.01}
Best R2 score :  0.942998255340148


In [23]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [1, 2, 3, 5, 10, 20, 30]
}
best_lasso = GridSearchCV(regressor, param_grid = params, cv = 5) # cv : the number of folds to be used for CV
best_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)

Grid search...


/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.261e+11, tolerance: 2.662e+09

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.813e+11, tolerance: 2.536e+09

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.607e+11, tolerance: 2.567e+09

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_des

...Done.
Best hyperparameters :  {'alpha': 30}
Best R2 score :  0.9482534443994037


/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.823e+11, tolerance: 2.651e+09

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.129e+11, tolerance: 2.566e+09

/usr/local/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.677e+11, tolerance: 3.247e+09



In [24]:
# Print R^2 scores
print("RIDGE / R2 score on training set : ", best_ridge.score(X_train, Y_train))
print("RIDGE / R2 score on test set : ", best_ridge.score(X_test, Y_test))
print()
print("LASSO / R2 score on training set : ", best_lasso.score(X_train, Y_train))
print("LASSO / R2 score on test set : ", best_lasso.score(X_test, Y_test))

RIDGE / R2 score on training set :  0.9809957527534967
RIDGE / R2 score on test set :  0.9551189058038105

LASSO / R2 score on training set :  0.9811540507189856
LASSO / R2 score on test set :  0.9552392315457086


In the end, after regulation, we can see that the best R2 score is with Lasso. 